In [ ]:
# do all necessary imports for this chapter
using LinearAlgebra
using TensorOperations
using TensorKit
using KrylovKit
include("TutorialFunctions.jl")
using .TutorialFunctions

# [Tangent-space methods for uniform matrix product states](https://doi.org/10.21468/SciPostPhysLectNotes.7)

## 0. Tensors in Julia

Throughout these tutorials we will make use of tensors as they usually appear throughout the physical sciences. Contrary to the Python tutorials where tensors are represented as plain numpy arrays, here we will not represent a tensor using a Julia `Base.Array` but rather make use of the `TensorMap` type exported by the Julia package [TensorKit.jl](https://github.com/Jutho/TensorKit.jl). In short, this package implements tensors as (multi)linear maps from a domain vectorspace to a codomain vectorspace. It provides all kinds of vector space operations (scalar multiplication, addition, norms and inner products), index operations (permutations) and linear algebra operations (multiplication, factorizations). In addition, tensor contractions can be performed using the `@tensor` macro from [TensorOperations.jl](https://github.com/Jutho/TensorOperations.jl).

For a detailed explanation on how to define and work with `TensorMap`s we refer to the [TensorKit.jl documentation](https://jutho.github.io/TensorKit.jl/latest/). Here, we will disregard any intricacies related to internal symmetries by restricting ourselves to (complex) Cartesian vector spaces of the sake of simplicity. This allows us to be a little less strict in working with tensor indices and helps keep the exposition of the algorithms clear without getting lost in details which are less important for this tutorial (at the cost of a one-time warning from the TensorKit internals). Sometimes we will be able to use the simpler `Tensor` type to represent a general tensor, which behaves much like you would expect a familiar multidimensional array. However, some situations greatly benefit from explicitly considering a tensor as a map from some some set of indices to some other set of indices. This should become clear as you progress through the tutorials, and we have added extra explanations for these use cases where possible.

## 1. Matrix product states in the thermodynamic limit

### 1.1 Normalization

We start by considering a uniform MPS in the thermodynamic limit, which is defined as 
$$\left | \Psi(A) \right \rangle = \sum_{\{i\}} \boldsymbol{v}_L^\dagger \left[ \prod_{m\in\mathbb{Z}} A^{i_m} \right] \boldsymbol{v}_R \left | \{i\} \right \rangle.$$

Here, $\boldsymbol{v}_L^\dagger$ and $\boldsymbol{v}_R$ represent boundary vectors at infinity and the $A^i$ are complex matrices of size $D \times D$ for every entry of the index $i$. This allows for the interpretation of the object $A$ as a three-legged tensor of dimensions $D\times d \times D$, where we will refer to $D$ as the bond dimension and $d$ as the physical dimension. With this object and the diagrammatic language of tensor networks, we can represent the state as

<center><img src="./img/umps.svg" alt="MPSstate"/></center>


Thus, we initialize and represent a uniform MPS state using a single `TensorMap` with three indices as follows:

In [ ]:
"""
Returns a random complex MPS tensor.

### Arguments

- `D::Int`: bond dimension for MPS.
- `d::Int`: physical dimension for MPS.

### Returns

`A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs, ordered left-bottom-right.
"""
function createMPS(D, d)
    A = Tensor(randn, ComplexF64, ℝ^D ⊗ ℝ^d ⊗ ℝ^D)
    return A
end;

In [ ]:
d, D = 3, 5
A = createMPS(D, d)

@assert A isa TensorMap{CartesianSpace} "Generated MPS tensor should be a TensorMap with Cartesian indices."
@assert map(i -> dim(space(A, i)), 1:numind(A)) == [D, d, D] "Generated MPS tensor has incorrect shape."
@assert storagetype(A) == Matrix{ComplexF64} "MPS tensor should have complex values"

One of the central objects in any MPS calculation is the transfer matrix, defined in our case as

<center><img src="./img/tm.svg" alt="transfer matrix" style="display=block; margin:auto"/></center>

This object corresponds to an operator acting on the space of $D \times D$ matrices, and can be interpreted as a 4-leg tensor. We will use the following convention for ordering the legs:
1. top left
2. bottom left
3. top right
4. bottom right

The transfer matrix can be shown to be a completely positive map, such that its leading eigenvalue is a positive number. This eigenvalue should be rescaled to one to ensure a proper normalization of the state in the thermodynamic limit. To perform this normalization, we must therefore find the left and right fixed points $l$ and $r$ which correspond to the largest eigenvalues of the eigenvalue equations

<center><img src="./img/fixedPoints.svg" alt="fixed points"/></center>

Normalizing the state then means rescaling the MPS tensor $A \leftarrow A / \sqrt{\lambda}$. Additionally, we may fix the normalization of the eigenvectors by requiring that their trace is equal to one:

<center><img src="./img/traceNorm.svg" alt="norm"/></center>

With these properties in place, the norm of an MPS can be evaluated as

<center><img src="./img/mpsNorm.svg" alt="norm"/></center>

It can be readily seen that the infinite product of transfer matrices reduces to a projector onto the fixed points, so that the norm reduces to the overlap between the boundary vectors and the fixed points. Since there is no effect of the boundary vectors on the bulk properties of the MPS, we can always choose these such that MPS is properly normalized as $ \left \langle \Psi(\bar{A})\middle | \Psi(A) \right \rangle = 1$.

One way of normalizing an MPS tensor is therefore by creating the transfer matrix as a `TensorMap`, and computing its leading eigenvectors and eigenvalues by interpreting it as an operator between sets of two indices depending on whether we want to compute the left of right fixed points. This can be done using the [`KrylovKit.eigsolve`](https://jutho.github.io/KrylovKit.jl/stable/man/eig/) method, which given a `TensorMap` and specified sets of 'left' and 'right' indices computes the eigenvectors and eigenvalues of the corresponding operator. Note that while the left and right fixed points $l$ and $r$ are naturally defined as (eigen)vectors, their normalization and positivity are more naturally characterized by interpreting them as matrices. Therefore, we will always interpret $l$ and $r$ as maps from their bottom to top and top to bottom indices respectively using the `TensorKit.permute` method on the vectors returned by `eigsolve`.

In [ ]:
"""
Form the transfermatrix of an MPS.

### Parameters

`A::TensorMap`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

-`E::TensorMap{CartesianSpace}``: Transfer matrix tensor with 4 legs of dimension (D, D, D, D), ordered topLeft-bottomLeft-topRight-bottomRight.
"""
function createTransfermatrix(A)
    @tensor E[-1 -2 -3 -4] := A[-1 1 -3] * A'[-2 1 -4]
    return E
end;

In [ ]:
"""
Normalize an MPS tensor.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `Anew::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Complexity

O(D^6) algorithm, diagonalizing (D^2, D^2) matrix.
"""
function normalizeMPS(A)
    E = createTransfermatrix(A)

    # calculate largest magnitude right eigenvalue by interpreting the transfer matrix as a
    # linear map from its right indices to its left indices
    vals, _ = eigsolve(E, (1, 2), (3, 4), 1, :LM)
    nrm = vals[1]
    Anew = A / sqrt(nrm)
    
    return Anew
end;

In [ ]:
"""
Find left fixed point.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `l::TensorMap{CartesianSpace, 1, 1}`: left fixed point with 2 legs of dimension (D, D), ordered bottom-top.

### Complexity

O(D^6) algorithm, diagonalizing (D^2, D^2) matrix.
"""
function leftFixedPoint(A)
    E = createTransfermatrix(A)
    
    # calculate largest magnitude left fixed point by interpreting the transfer matrix as a
    # linear map from its left indices to its right indices
    _, vecs, _ = eigsolve(E, (4, 3), (2, 1), 1, :LM)
    l = permute(vecs[1], (1,), (2,)) # interpret vector as matrix
    
    # make left fixed point hermitian and positive semidefinite explicitly
    tracel = tr(l)
    l /= (tracel / abs(tracel)) # remove possible phase
    l = (l + l') / 2 # force hermitian

    return l
end;

In [ ]:
"""
Find right fixed point.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `r::TensorMap{CartesianSpace, 1, 1}`: right fixed point with 2 legs of dimension (D, D), top-bottom.

### Complexity

O(D^6) algorithm, diagonalizing (D^2, D^2) matrix.
"""
function rightFixedPoint(A)
    E = createTransfermatrix(A)
    
    # calculate largest magnitude right fixed point by interpreting the transfer matrix as a
    # linear map from its right indices to its left indices
    _, vecs, _ = eigsolve(E, (1, 2), (3, 4), 1, :LM)
    r = permute(vecs[1], (1,), (2,)) # interpret vector as matrix
    
    # make right fixed point hermitian and positive semidefinite explicitly
    tracer = tr(r)
    r /= (tracer / abs(tracer)) # remove possible phase
    r = (r + r') / 2 # force hermitian
    
    return r
end;

In [ ]:
"""
Find normalized fixed points.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `l::TensorMap{CartesianSpace, 1, 1}`: left fixed point with 2 legs of dimension (D, D), bottom-top.
- `r::TensorMap{CartesianSpace, 1, 1}`: right fixed point with 2 legs of dimension (D, D), top-bottom.

### Complexity

O(D^6) algorithm, diagonalizing (D^2, D^2) matrix.
"""
function fixedPoints(A)
    l, r = leftFixedPoint(A), rightFixedPoint(A)
    
    # calculate trace
    trace = tr(l * r)
    
    return l / trace, r
end;

We can then perform some simple checks to verify that these methods indeed return the properly normalized left and right fixed points. However, at this point, a remark regarding the use of the `@tensor` macro with `TensorMap`s is in order. As stated above, we explicitly construct $l$ and $r$ as `TensorMap`s of rank (1, 1). When working with `TensorMap`s with non-trivial codomain, it is often desirable to explicitly indicate this in the `@tensor` notation, both to avoid accidental space mismatches and to be more explicit about the nature of the tensors involved. This specification is done by adding a `;` between indices in the domain and the codomain, as illustrated below. We try to systematically adhere to this specification as we will naturally encounter more `TensorMap`s with non-trivial partitionings between domain and codomain, for example when considering canonical forms below.

In [ ]:
A = normalizeMPS(A)
l, r = fixedPoints(A)

@assert l isa TensorMap{CartesianSpace, 1, 1} "l should be a TensorMap of rank (1, 1)"
@assert r isa TensorMap{CartesianSpace, 1, 1} "r should be a TensorMap of rank (1, 1)"
@assert l ≈ l' "left fixed point should be hermitian!"
@assert r ≈ r' "right fixed point should be hermitian!"

@tensor lp[-1; -2] := A[1 2 -2] * l[3; 1] * conj(A[3 2 -1])
@tensor rp[-1; -2] := A[-1 2 1] * r[1; 3] * conj(A[-2 2 3])
tracelr = tr(l * r)

@assert l ≈ lp "l should be a left fixed point!"
@assert r ≈ rp "r should be a right fixed point!"
@assert abs(tracelr - 1) < 1e-12 "Left and right fixed points should be trace normalized!"

### 1.2 Gauge freedom
While a given MPS tensor $A$ corresponds to a unique state $\left | \Psi(A) \right \rangle$, the converse is not true, as different tensors may give rise to the same state. This is easily seen by noting that the gauge transform

<center><img src="img/gaugeTransform.svg" alt="gauge transform"></center>

leaves the physical state invariant. We may use this freedom in parametrization to impose canonical forms on the MPS tensor $A$.

We start by considering the *left-orthonormal form* of an MPS, which is defined in terms of a tensor $A_L$ that satisfies the condition

<center><img src="img/leftOrth.svg" alt="left orthonormal"></center>

We can find the gauge transform $L$ that brings $A$ into this form

<center><img src="img/leftGauge.svg" alt="left gauge"></center>

by decomposing the fixed point $l$ as $l = L^\dagger L$, such that

<center><img src="img/leftOrth2.svg" alt="left orthonormal2"></center>

Note that this gauge choice still leaves room for unitary gauge transformations

<center><img src="img/unitaryGauge.svg" alt="unitary gauge"></center>

which can be used to bring the right fixed point $r$ into diagonal form. Similarly, we can find the gauge transform that brings $A$ into *right-orthonormal form*

<center><img src="img/rightGauge.svg" alt="right gauge"></center>

such that

<center><img src="img/rightOrth.svg" alt="right gauge"></center>

and the left fixed point $l$ is diagonal. This way of bringing a given MPS into canonical form by decomposing the corresponding transfer matrix fixed points can be implemented using the matrix square root `sqrt` for `TensorMap`s as follows:

In [ ]:
"""
Transform A to left-orthonormal gauge.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `L::TensorMap{CartesianSpace, 1, 1}`: left gauge tensor with 2 legs of dimension (D, D), ordered left-right.
- `Al::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, left orthonormal.

### Complexity

O(D^6) algorithm, diagonalizing (D^2, D^2) matrix.
"""
function leftOrthonormalize(A, l=leftFixedPoint(A))
    # decompose l = L' * L using matrix square root
    L = sqrt(l)
    
    # apply gauge L to A
    @tensor Al[-1 -2 -3] := L[-1; 1] * A[1 -2 2] * inv(L)[2; -3]

    return L, Al
end;

In [ ]:
"""
Transform A to right-orthonormal gauge.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `R::TensorMap{CartesianSpace, 1, 1}`: right gauge tensor with 2 legs of dimension (D, D), ordered left-right.
- `Ar::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, right orthonormal.

### Complexity

O(D^6) algorithm, diagonalizing (D^2, D^2) matrix.
"""
function rightOrthonormalize(A, r=rightFixedPoint(A))
    # decompose r = R * R' using matrix square root
    R = sqrt(r)
    
    # apply gauge R to A
    @tensor Ar[-1 -2 -3] := inv(R)[-1; 1] * A[1 -2 2] * R[2; -3]

    return R, Ar
end;

In [ ]:
L, Al = leftOrthonormalize(A, l)
R, Ar = rightOrthonormalize(A, r)

@assert L isa TensorMap{CartesianSpace, 1, 1} "L should be a TensorMap of rank (1, 1)"
@assert R isa TensorMap{CartesianSpace, 1, 1} "R should be a TensorMap of rank (1, 1)"
@assert L' * L ≈ l "Left gauge doesn't sqaure to l"
@assert R * R' ≈ r "Right gauge doesn't square to r"

# check if left and right canonical tensors contract to the identity map
@tensor Al_id[-1; -2] := Al[1 2 -2] * conj(Al[1 2 -1])
@tensor Ar_id[-1; -2] := Ar[-1 1 2] * conj(Ar[-2 1 2])

@assert Ar_id ≈ id(space(Ar, 1)) "Ar not in right-orthonormal form"
@assert Al_id ≈ id(space(Al, 3)) "Al not in left-orthonormal form"

Finally, we can define a *mixed gauge* for the uniform MPS by choosing one site, the 'center site', and bringing all tensors to the left of it in the left-orthonormal form and all the tensors to the right of it in the right-orthonormal form. Defining a new tensor $A_C$ on the center site, we obtain the form

<center><img src="img/mixedGauge.svg" alt="right gauge"></center>

By contrast, the original representation using the same tensor at every site is commonly referred to as the *uniform gauge*. The mixed gauge has an intuitive interpretation. Defining $C = LR$, this tensor then implements the gauge transform that maps the left-orthonormal tensor to the right-orthonormal one, thereby defining the center-site tensor $A_C$:

<center><img src="img/mixedGauge2.svg" alt="right gauge"></center>

This relation is called the mixed gauge condition and allows us to freely move the center tensor $A_C$ through the MPS, linking the left- and right orthonormal tensors.

Finally we may bring $C$ into diagonal form by performing a singular value decomposition $C = USV^\dagger$ (which you can easily do using [`TensorKit.tsvd`](https://jutho.github.io/TensorKit.jl/latest/lib/tensors/#TensorKit.tsvd)) and absorbing $U$ and $V^\dagger$ into the definition of $A_L$ and $A_R$ using the residual unitary gauge freedom

<center><img src="img/diagC.svg" alt="mixed gauge3"></center>

The mixed canonical form with a diagonal $C$ now allows to straightforwardly write down a Schmidt decomposition of the state across an arbitrary bond in the chain

$$ \left | \Psi(A) \right \rangle = \sum_{i=1}^{D} C_i \left | \Psi^i_L(A_L) \right \rangle \otimes \left | \Psi^i_R(A_R) \right \rangle,$$

where the states $\left | \Psi^i_L(A_L) \right \rangle$ and $\left | \Psi^i_R(A_R) \right \rangle$ are orthogonal states on half the lattice. The diagonal elements $C_i$ are exactly the Schmidt numbers of any bipartition of the MPS, and as such determine its bipartite entanglement entropy

$$ S = -\sum_i C_i^2 \log(C_i^2) .$$

In [ ]:
"""
Bring MPS tensor into mixed gauge, such that -Al-C- = -C-Ar- = Ac.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `Al::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, left orthonormal.
- `Ac::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, center gauge.
- `Ar::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, right orthonormal.
- `C::TensorMap{CartesianSpace, 1, 1}`: center gauge tensor with 2 legs of dimension (D, D), ordered left-right, diagonal.

### Complexity

O(D^6) algorithm, diagonalizing (D^2, D^2) matrix.
"""
function mixedCanonical(A)

    # Compute left and right orthonormal forms
    L, Al = leftOrthonormalize(A)
    R, Ar = rightOrthonormalize(A)
    
    # center matrix C is matrix multiplication of L and R
    C = L * R
    
    # singular value decomposition to diagonalize C
    U, C, V = tsvd(C, (1,), (2,))

    # absorb corresponding unitaries in Al and Ar
    @tensor Al[-1 -2 -3] = U'[-1; 1] * Al[1 -2 2] * U[2; -3]
    @tensor Ar[-1 -2 -3] = V[-1; 1] * Ar[1 -2 2] * V'[2; -3]

    # normalize center matrix
    norm = tr(C * C')
    C /= sqrt(norm)

    # compute center MPS tensor
    @tensor Ac[-1 -2 -3] := Al[-1 -2 1] * C[1; -3]

    return Al, Ac, Ar, C
end;

In [ ]:
"""
Calculate the entanglement spectrum of an MPS.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `S::Vector{Float64}`: vector containing the singular values of the center matrix, representing the entanglement spectrum.
- `entropy::Float64`: entanglement entropy across a leg.
"""
function entanglementSpectrum(A)
    # go to mixed gauge
    _, _, _, C = mixedCanonical(A)

    # calculate entropy
    S = diag(C[])
    entropy = -sum(sum(s.^2 .* log.(s.^2) for s in S))
    
    return S, entropy
end;

In [ ]:
# check that gauging is still correct
Al, Ac, Ar, C = mixedCanonical(A)
S, entropy = entanglementSpectrum(A)

@tensor Ar_id[-1; -2] := Ar[-1 1 2] * conj(Ar[-2 1 2])
@tensor Al_id[-1; -2] := Al[1 2 -2] * conj(Al[1 2 -1])
@tensor LHS[-1 -2 -3] := Al[-1 -2 1] * C[1; -3]
@tensor RHS[-1 -2 -3] := C[-1; 1] * Ar[1 -2 -3]

@assert C isa TensorMap{CartesianSpace, 1, 1} "C should be a TensorMap of rank (1, 1)"
@assert Ar_id ≈ id(space(Ar, 1)) "Ar not in right-orthonormal form"
@assert Al_id ≈ id(space(Al, 3)) "Al not in left-orthonormal form"
@assert LHS ≈ RHS && RHS ≈ Ac "Something went wrong in gauging the MPS"

### 1.3 Truncation of a uniform MPS

The mixed canonical form also enables efficient truncatation of an MPS. The sum in the above Schmidt decomposition can be truncated, giving rise to a new MPS that has a reduced bond dimension for that bond. This truncation is optimal in the sense that the norm between the original and the truncated MPS is maximized. To arrive at a translation invariant truncated MPS, we can truncate the columns of the absorbed isometries $U$ and $V^\dagger$ correspondingly, thereby transforming *every* tensor $A_L$ or $A_R$. The truncated MPS in the mixed gauge is then given by

<center><img src="img/truncMPS.svg" alt="truncated MPS"></center>

We note that the resulting state based on this local truncation is not guaranteed to correspond to the MPS with a lower bond dimension that is globally optimal. This would require a variational optimization of the cost function.

$$ \left | \left | ~\left | \Psi(A) \right \rangle - \left | \Psi(\tilde{A}) \right \rangle ~\right | \right |^2.$$

This truncation can be performed using the `trunc` keyword argument in [`TensorKit.tsvd`](https://jutho.github.io/TensorKit.jl/latest/lib/tensors/#TensorKit.tsvd).

In [ ]:
"""
Truncate an MPS to a lower bond dimension.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `Ãl::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (Dtrunc, d, Dtrunc), ordered left-bottom-right, left orthonormal.
- `Ãc::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (Dtrunc, d, Dtrunc), ordered left-bottom-right, center gauge.
- `Ãr::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (Dtrunc, d, Dtrunc), ordered left-bottom-right, right orthonormal.
- `C̃::TensorMap{CartesianSpace, 1, 1}`: center gauge tensor with 2 legs of dimension (Dtrunc, Dtrunc), ordered left-right, diagonal.
"""
function truncateMPS(A, Dtrunc)
    Al, Ac, Ar, C = mixedCanonical(A)
    
    # perform SVD and truncate:
    U, S, V = tsvd(C, (1,), (2,); trunc=truncdim(Dtrunc))
    
    # reabsorb unitaries
    @tensor Ãl[-1 -2 -3] := U'[-1; 1] * Al[1 -2 2] * U[2; -3]
    @tensor Ãr[-1 -2 -3] := V[-1; 1] * Ar[1 -2 2] * V'[2; -3]
    C̃ = S
    
    # normalize center matrix
    norm = tr(C̃ * C̃')
    C̃ /= sqrt(norm)

    # compute center MPS tensor
    @tensor Ãc[-1 -2 -3] := Ãl[-1 -2 1] * C̃[1; -3]
    
    return Ãl, Ãc, Ãr, C̃
end;

In [ ]:
Dtrunc = 3
Ãl, Ãc, Ãr, C̃ = truncateMPS(A, Dtrunc)
@assert dim(space(Ãl, 1)) == Dtrunc && dim(space(Ãl, 3)) == Dtrunc

### 1.4 Algorithms for finding canonical forms
The success of using MPS for describing physical systems stems from the fact that they provide efficient approximations to a large class of physically relevant states. In one dimension, they have been shown to approximate low-energy states of gapped systems arbitrarily well at only a polynomial cost in the bond dimension $D$. This means that in principle we can push MPS results for these systems to arbitrary precision as long as we increase the bond dimension enough. However, increasing the bond dimension comes at a numerical cost, as the complexity of any MPS algorithm scales with $D$. As opposed to the naive routines given above, it is possible to ensure that the complexity of all MPS algorithms scales as $O(D^3)$, so long as we are a bit careful when implementing them.

As a first example, we can refrain from explicitly contructing the matrices that are used in the eigenvalue problems, and instead pass a function that implements the action of the corresponding operator on a vector to the eigenvalue solver. We demonstrate this for the problem of normalizing an MPS, where instead of explicitly constructing the transfer matrix we now provide a function which implements its action on the right and left fixed points using an optimal contraction sequence:

In [ ]:
"""
Normalize an MPS tensor.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `Anew::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Complexity

O(D^3) algorithm, D^3 contraction for transfer matrix handle.
"""
function normalizeMPS(A)
    vals, _, _ =
        eigsolve(TensorMap(randn, scalartype(A), space(A, 1) ← space(A, 1)), 1, :LM) do v
            @tensor vout[-1; -2] := A[-1 2 1] * conj(A[-2 2 3]) * v[1; 3]
        end

    Anew = A / sqrt(vals[1])

    return Anew
end;

In [ ]:
"""
Find left fixed point.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `l::TensorMap{CartesianSpace, 1, 1}`: left fixed point with 2 legs of dimension (D, D), ordered bottom-top.

### Complexity

O(D^3) algorithm, D^3 contraction for transfer matrix handle.
"""
function leftFixedPoint(A)
    # calculate fixed point
    _, vecs, _ =
        eigsolve(TensorMap(randn, scalartype(A), space(A, 1) ← space(A, 1)), 1, :LM) do v
            @tensor vout[-1; -2] := A[1 2 -2] * conj(A[3 2 -1]) * v[3; 1]
        end
    l = vecs[1]
    
    # make left fixed point hermitian and positive semidefinite explicitly
    tracel = tr(l)
    l /= (tracel / abs(tracel)) # remove possible phase
    l = (l + l') / 2 # force hermitian

    return l
end;

In [ ]:
"""
Find right fixed point.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `r::TensorMap{CartesianSpace, 1, 1}`: right fixed point with 2 legs of dimension (D, D), ordered top-bottom.

### Complexity

O(D^3) algorithm, D^3 contraction for transfer matrix handle.
"""
function rightFixedPoint(A)
    # calculate fixed point
    _, vecs, _ =
        eigsolve(TensorMap(randn, scalartype(A), space(A, 1) ← space(A, 1)), 1, :LM) do v
            @tensor vout[-1; -2] := A[-1 2 1] * conj(A[-2 2 3]) * v[1; 3]
        end
    r = vecs[1]
    
    # make right fixed point hermitian and positive semidefinite explicitly
    tracer = tr(r)
    r /= (tracer / abs(tracer)) # remove possible phase
    r = (r + r') / 2 # force hermitian

    return r
end;

In [ ]:
"""
Find normalized fixed points.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `l::TensorMap{CartesianSpace, 1, 1}`: left fixed point with 2 legs of dimension (D, D), bottom-top.
- `r::TensorMap{CartesianSpace, 1, 1}`: right fixed point with 2 legs of dimension (D, D), top-bottom.

### Complexity

O(D^3) algorithm, D^3 contraction for transfer matrix handle.
"""
function fixedPoints(A)
    # find fixed points
    l, r = leftFixedPoint(A), rightFixedPoint(A)

    # calculate trace
    trace = tr(l * r)

    return l / trace, r
end;

In [ ]:
A = createMPS(D, d)
A = normalizeMPS(A)
l, r = fixedPoints(A)

@assert l isa TensorMap{CartesianSpace, 1, 1} "l should be a TensorMap of rank (1, 1)"
@assert r isa TensorMap{CartesianSpace, 1, 1} "r should be a TensorMap of rank (1, 1)"
@assert l ≈ l' "left fixed point should be hermitian!"
@assert r ≈ r' "right fixed point should be hermitian!"

@tensor lp[-1; -2] := A[1 2 -2] * l[3; 1] * conj(A[3 2 -1])
@tensor rp[-1; -2] := A[-1 2 1] * r[1; 3] * conj(A[-2 2 3])
tracelr = tr(l * r)

@assert l ≈ lp "l should be a left fixed point!"
@assert r ≈ rp "r should be a right fixed point!"
@assert abs(tracelr - 1) < 1e-12 "Left and right fixed points should be trace normalized!"

We can similarly improve both the efficiency and accuracy of the routines bringing a given MPS into its mixed canonical form. While plugging in the more efficient ways of finding the left and right fixed point into the above `mixedCanonical` routine would reduce its complexity to $O(D^3)$, this algorithm would still be suboptimal in terms of numerical accuracy. This arises from the fact that, while $l$ and $r$ are theoretically known to be positive hermitian matrices, at least one of them will nevertheless have small eigenvalues, say of order $\eta$, if the MPS is supposed to provide a good approximation to an actual state. In practice, $l$ and $r$ are determined using an iterative eigensolver and will only be accurate up to a specified tolerance $\epsilon$. Upon taking the 'square roots' $L$ and $R$, the numerical precision will then decrease to $\text{min}(\sqrt{\epsilon}, \epsilon / \sqrt{\eta})$. Furthermore, gauge transforming $A$ with $L$ or $R$ requires the potentially ill-conditioned inversions of $L$ and $R$, and will typically yield $A_L$ and $A_R$ which violate the orthonormalization condition in the same order $\epsilon/\sqrt{\eta}$. We can circumvent both these probelems by resorting to so-called *single-layer algorithms*. These are algorithms that only work on the level of the MPS tensors in the ket layer, and never consider operations for which contractions with the bra layer are needed. We now demonstrate such a single-layer algorithm for finding canonical forms.

Suppose we are given an MPS tensor $A$, then from the above discussion we know that bringing it into left canonical form means finding a left-orthonormal tensor $A_L$ and a matrix $L$ such that $L A=A_L L$. The idea is then to solve this equation iteratively, where in every iteration

1. we start from a matrix $L^{i}$
2. we construct the tensor $L^{i}A$
3. we take a QR decomposition to obtain $A_L^{i+1} L^{i+1} = L^{i}A$, and
4. we take $L^{i+1}$ to the next iteration

The QR decomposition is represented diagrammatically as

<center><img src="img/qrStep.svg" alt="QR step"></center>

This iterative procedure is bound to converge to a fixed point for which $L^{(i+1)}=L^{(i)}=L$ and $A_L$ is left orthonormal by construction:

<center><img src="img/qrConv.svg" alt="QR convergence"></center>

A similar procedure can be used to find a right-orthonormal tensor $A_R$ and a matrix $R$ such that $A R = R A_R$. It is important to note that the convergence of this procedure relies on the fact that the QR decomposition is unique, which is not actually the case in general. However, it can be made unique by imposing that the diagonal elements of the triangular matrix $R$ must be positive. This extra condition is automatically imposed by using the `TensorKit.leftorth` and `TensorKit.rightorth` routines using the keyword arguments `alg=QRpos()` and `alg=LQpos()` respectively.

Finally, we note that using `TensorKit.leftorth` and `TensorKit.rightorth` in this way naturally gives the left isometry $A_L$ as a rank (2, 1) `TensorMap` and the right isometry $A_R$ as a rank (1, 2) `TensorMap`, contrary to the previous implementation where there was not really a natural partition between domain and codomain for $A_L$ and $A_R$. From here on out, we will therefore adhere to this natural partitioning and always take the rank of $A_L$ and $A_R$ into account explicitly.

In [ ]:
"""
Transform A to right-orthonormal gauge.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.
- `R0::TensorMap{CartesianSpace, 1, 1}`: right gauge tensor with 2 legs of dimension (D, D), initial guess.
- `tol::Float64=1e-14`: convergence criterium, `norm(R - Rnew) < tol`.
- `maxIter::Int=1e5`: maximum amount of iterations.

### Returns

- `R::TensorMap{CartesianSpace, 1, 1}`: right gauge tensor with 2 legs of dimension (D, D), ordered left-right.
- `Ar::TensorMap{CartesianSpace, 1, 2}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, right orthonormal.
"""
function rightOrthonormalize(
    A, R0=TensorMap(randn, scalartype(A), space(A, 1) ← space(A, 3)); tol=1e-14, maxIter=1e5
)
    tol = max(tol, 1e-14)
    i = 1

    # Normalize R0
    R0 /= norm(R0)

    # Initialize loop
    @tensor Ai[-1 -2 -3] := A[-1 -2 1] * R0[1; -3]
    R, Ar = rightorth(Ai, (1,), (2, 3); alg=LQpos())
    R /= norm(R)
    convergence = norm(R - R0)

    # Decompose A*R until R converges
    while convergence > tol
        # calculate AR and decompose
        @tensor Ai[-1 -2 -3] = A[-1 -2 1] * R[1; -3]
        Rnew, Ar = rightorth(Ai, (1,), (2, 3); alg=LQpos())

        # normalize new R
        Rnew /= norm(Rnew)

        # calculate convergence criterium
        convergence = norm(Rnew - R)
        R = Rnew

        # check if iterations exceeds maxIter
        if i > maxIter
            println("Warning, right decomposition has not converged ", convergence)
            break
        end
        i += 1
    end

    return R, Ar
end;

In [ ]:
"""
Transform A to left-orthonormal gauge.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.
- `L0::TensorMap{CartesianSpace, 1, 1}`: left gauge tensor with 2 legs of dimension (D, D), initial guess.
- `tol::Float64=1e-14`: convergence criterium, `norm(R - Rnew) < tol`.
- `maxIter::Int=1e5`: maximum amount of iterations.

### Returns

- `L::TensorMap{CartesianSpace, 1, 1}`: left gauge tensor with 2 legs of dimension (D, D), ordered left-right.
- `Al::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, left orthonormal.
"""
function leftOrthonormalize(
    A, L0=TensorMap(randn, scalartype(A), space(A, 1) ← space(A, 3)); tol=1e-14, maxIter=1e5
)
    tol = max(tol, 1e-14)
    i = 1

    # Normalize L0
    L0 /= norm(L0)

    # Initialize loop
    @tensor Ai[-1 -2 -3] := L0[-1; 1] * A[1 -2 -3]
    Al, L = leftorth(Ai, (1, 2), (3,); alg=QRpos())
    L /= norm(L)
    convergence = norm(L - L0)

    # Decompose L*A until L converges
    while convergence > tol
        # calculate LA and decompose
        @tensor Ai[-1 -2 -3] = L[-1; 1] * A[1 -2 -3]
        Al, Lnew = leftorth(Ai, (1, 2), (3,); alg=QRpos())

        # normalize new L
        Lnew /= norm(Lnew)

        # calculate convergence criterium
        convergence = norm(Lnew - L)
        L = Lnew

        # check if iterations exceeds maxIter
        if i > maxIter
            println("Warning, left decomposition has not converged ", convergence)
            break
        end
        i += 1
    end
    
    return L, Al
end;

In [ ]:
"""
Bring MPS tensor into mixed gauge, such that -Al-C- = -C-Ar- = Ac.

### Arguments

- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.

### Returns

- `Al::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, left orthonormal.
- `Ac::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, center gauge.
- `Ar::TensorMap{CartesianSpace, 1, 2}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, right orthonormal.
- `C::TensorMap{CartesianSpace, 1, 1}`: center gauge tensor with 2 legs of dimension (D, D), ordered left-right, diagonal.

### Complexity

O(D^3) algorithm.
"""
function mixedCanonical(
    A;
    L0=TensorMap(randn, scalartype(A), space(A, 1) ← space(A, 3)),
    R0=TensorMap(randn, scalartype(A), space(A, 1) ← space(A, 3)),
    tol=1e-14,
    maxIter=1e5,
)
    tol = max(tol, 1e-14)

    # Compute left and right orthonormal forms
    L, Al = leftOrthonormalize(A, L0; tol, maxIter)
    R, Ar = rightOrthonormalize(A, R0; tol, maxIter)

    # center matrix C is matrix multiplication of L and R
    C = L * R

    # singular value decomposition to diagonalize C
    U, C, V = tsvd(C, (1,), (2,))

    # absorb corresponding unitaries in Al and Ar
    @tensor Al[-1 -2; -3] = U'[-1; 1] * Al[1 -2; 2] * U[2; -3]
    @tensor Ar[-1; -2 -3] = V[-1; 1] * Ar[1; -2 2] * V'[2; -3]

    # normalize center matrix
    norm = tr(C * C')
    C /= sqrt(norm)

    # compute center MPS tensor
    @tensor Ac[-1 -2; -3] := Al[-1 -2; 1] * C[1; -3]

    return Al, Ac, Ar, C
end;

In [ ]:
Al, Ac, Ar, C = mixedCanonical(A)

@assert Al isa TensorMap{CartesianSpace, 2, 1} "Al should be a TensorMap of rank (2, 1)"
@assert Ac isa TensorMap{CartesianSpace, 2, 1} "Ac should be a TensorMap of rank (2, 1)"
@assert Ar isa TensorMap{CartesianSpace, 1, 2} "Ar should be a TensorMap of rank (1, 2)"
@assert C isa TensorMap{CartesianSpace, 1, 1} "C should be a TensorMap of rank (1, 1)"

@tensor Ar_id[-1; -2] := Ar[-1; 1 2] * conj(Ar[-2; 1 2])
@tensor Al_id[-1; -2] := Al[1 2; -2] * conj(Al[1 2; -1])
@tensor LHS[-1 -2; -3] := Al[-1 -2; 1] * C[1; -3]
@tensor RHS[-1 -2; -3] := C[-1; 1] * Ar[1; -2 -3]

@assert Ar_id ≈ id(space(Ar, 1)) "Ar not in right-orthonormal form"
@assert Al_id ≈ id(space(Al, 3)) "Al not in left-orthonormal form"
@assert LHS ≈ RHS && RHS ≈ Ac "Something went wrong in gauging the MPS"

#### Note:
For the remainder of this tutorial, keep in mind that you should always aim to reduce the complexity of your algorithm to $O(D^3)$ in order to keep the computational cost tractable.

### 1.5 Computing expectation values 
Now that we have seen the different ways to parametrize a given MPS, namely the uniform gauge and the mixed gauge, we wish to use these to compute expectation values of an extensive operator:
$$ O = \frac{1}{\mathbb{Z}} \sum_{n \in \mathbb{Z}} O_n. $$

If we assume that each $O_n$ acts on a single site and we are working with a properly normalized MPS, translation invariance dictates that the expectation value of $O$ is given by the contraction

<center><img src="img/expVal.svg" alt="Expectation value"></center>

In the uniform gauge, we can use the fixed points of the transfer matrix to contract everything to the left and to the right of the operator, such that we are left with the contraction

<center><img src="img/expVal2.svg" alt="Expectation value 2"></center>

In the mixed gauge, we can locate the center site where the operator is acting, and then contract everything to the left and right to the identity to arrive at the particularly simple expression

<center><img src="img/expVal3.svg" alt="Expectation value 3"></center>

In [ ]:
"""
Calculate the expectation value of a 1-site operator in uniform gauge.

### Arguments

- `O::TensorMap{CartesianSpace, 1, 1}`: single-site operator with 2 legs of dimension (d, d), ordered top-bottom.
- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.
- `fpts::Tuple=fixedPoints(A)`: left and right fixed points of transfermatrix, normalized.

### Returns

- `o::ComplexF64`: expectation value of `O`.
"""
function expVal1Uniform(O, A, fpts=fixedPoints(A))
    l, r = fpts
    # contract expectation value network
    @tensor o = l[4 1] * r[3 6] * A[1 2 3] * conj(A[4 5 6]) * O[2; 5]

    return o
end;

In [ ]:
"""
Calculate the expectation value of a 1-site operator in mixed gauge.

### Arguments

- `O::TensorMap{CartesianSpace, 1, 1}`: single-site operator with 2 legs of dimension (d, d), ordered top-bottom.
- `Ac::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, center gauged.

### Returns

- `o::ComplexF64`: expectation value of `O`.
"""
function expVal1Mixed(O, Ac)
    # contract expectation value network
    @tensor o = Ac[2 1; 3] * conj(Ac[2 4; 3]) * O[1; 4]

    return o
end;

In [ ]:
O = TensorMap(randn, ComplexF64, ℝ^d ← ℝ^d)
A = createMPS(D, d)
A = normalizeMPS(A)
Al, Ac, Ar, C = mixedCanonical(A)
expVal = expVal1Uniform(O, A)
expValMix = expVal1Mixed(O, Ac)
diff = abs(expVal - expValMix)
@assert diff < 1e-12 "different gauges give different values?"

This procedure can be readily generalized to operators that act on multiple sites. In particular, a two-site operator such as a Hamiltonian term $h$ can be evaluated as

<center><img src="img/expValHam.svg" alt="Expectation value Hamiltonian"></center>

In [ ]:
"""
Calculate the expectation value of a 2-site operator in uniform gauge.

### Arguments

- `O::TensorMap{CartesianSpace, 2, 2}`: two-site operator with 4 legs of dimension (d, d, d, d), ordered topLeft-topRight-bottomLeft-bottomRight.
- `A::TensorMap{CartesianSpace}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.
- `fpts::Tuple=fixedPoints(A)`: left and right fixed points of transfermatrix, normalized.

### Returns

- `o::ComplexF64`: expectation value of `O`.
"""
function expVal2Uniform(O, A, fpts=fixedPoints(A))
    l, r = fpts
    # contract expectation value network
    @tensor o = l[6; 1] * r[5; 10] * A[1 2 3] * A[3 4 5] * conj(A[6 7 8]) * conj(A[8 9 10]) * O[2 4; 7 9]

    return o
end;

In [ ]:
"""
Calculate the expectation value of a 2-site operator in mixed gauge.

### Arguments

- `O::TensorMap{CartesianSpace, 2, 2}`: two-site operator with 4 legs of dimension (d, d, d, d), ordered topLeft-topRight-bottomLeft-bottomRight.
- `Ac::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, center gauged.
- `Ar::TensorMap{CartesianSpace, 1, 2}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, right gauged.

### Returns

- `o::ComplexF64`: expectation value of `O`.
"""
function expVal2Mixed(O, Ac, Ar)
    # contract expectation value network
    @tensor o = Ac[4 2; 1] * Ar[1; 3 6] * conj(Ac[4 5; 8]) * conj(Ar[8; 7 6]) * O[2 3; 5 7]

    return o
end;

In [ ]:
O2 = TensorMap(randn, ComplexF64, ℝ^d ⊗ ℝ^d ← ℝ^d ⊗ ℝ^d)

expVal = expVal2Uniform(O2, A)
expValGauge = expVal2Mixed(O2, Ac, Ar)
expValGauge2 = expVal2Mixed(O2, Al, Ac)

diff1 = abs(expVal - expValGauge)
diff2 = abs(expVal - expValGauge2)
@assert diff1 < 1e-12 && diff2 < 1e-12 "different gauges give different values?"